# 自动微分

假设我们想对函数$y=2\mathbf{x}^{\top}\mathbf{x}$关于列向量$\mathbf{x}$求导

In [9]:
import torch

x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

在我们计算$y$关于$\mathbf{x}$的梯度之前，我们需要一个地方来存储梯度

In [10]:
x.requires_grad_(True) #equivalent to `x=torch.arange(4.0,requires_grad = True)`
x.grad

现在让我们计算$y$

In [11]:
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

通过调用反向传播函数来自动计算`y`关于`x`每个分量的梯度

In [12]:
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

In [13]:
x.grad == 4 * x

tensor([True, True, True, True])

现在让我们计算`x`的另一个函数

In [15]:
x.grad.zero_()#默认情况下PyTorch会累积梯度，因此需要清除之前的值([1,5,9,13])
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

深度学习中
，我们的目的不是计算微分矩阵，而是单独计算批量中每个样本的偏导数之和

In [7]:
x.grad.zero_()
y = x * x#y是一个向量，通常我们不进行向量对向量的求导，而将其通过加和转化成标量对向量的求导
#等价于y.backward(torch.ones(len(x)))
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

将某些计算移动到记录的计算图之外

In [17]:
x.grad.zero_()
y = x * x
u = y.detach()#把y转化成常数u，而不是x的函数
z = u * x

z.sum().backward()
x.grad == u

tensor([True, True, True, True])

In [18]:
u

tensor([0., 1., 4., 9.])

In [21]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

即使构建函数的计算图需要通过Python控制流（例如，条件、循环或任意函数调用），我们仍然可以计算得到的变量的梯度

隐式构造：对控制流好一些，但慢一些

In [22]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

a = torch.randn(size=(), requires_grad=True)#size=()——标量
d = f(a)
d.backward()

a.grad == d / a

tensor(True)

In [23]:
a.grad,d/a

(tensor(204800.), tensor(204800., grad_fn=<DivBackward0>))